In [1]:
import pickle
import torch
import cv2
from torch.utils.data import Dataset, DataLoader
import numpy as np
#https://pytorch.org/hub/ultralytics_yolov5/

# with open("CUDatamap_line2.pkl","wb") as f:
#     pickle.dump(data_line_map,f) #Store the datamap for further processig
# with open("CUDatamap_coef.pkl","wb") as f:
#     pickle.dump(data_coef_map,f) #Store the datamap for further processig

with open("CUDatamap_line2.pkl","rb") as f:
    data_line_map = pickle.load(f)
with open("CUDatamap_stline2.pkl","rb") as f:
    data_stline_map = pickle.load(f)

with open("CUDatamap_coef.pkl","rb") as f:
    data_coef_map = pickle.load(f)



y_col = np.linspace(400,580,19)

class LaneDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_line_map, scale=None):
        
        self.new_data_keys = list(data_line_map.keys())
        self.data_line_map = data_line_map
        self.scale = scale
  

    def __len__(self):
        return len(self.new_data_keys)
        

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_loc = self.new_data_keys[idx]
        lines = data_line_map[image_loc] #Since
        image = cv2.imread(image_loc)
        stlines = data_stline_map[image_loc]

        return (image,lines,stlines,idx)

In [2]:
lane_dataset = LaneDataset(data_line_map=data_line_map)
for sample in lane_dataset:
    image,lines,st_lines,idx = sample
    break
lane_dataset = LaneDataset(data_line_map=data_line_map)

In [3]:
height,width,channel = image.shape
print(height,width,channel)

def get_lane_angle(myline):
    global y_col #its actually row
    row = y_col[-1]
    col = myline[-1]
    
    new_col = col - width//2
    new_row = row + height

    angle = -np.arctan2(new_col,new_row)
    # if angle<0:
    #     angle = 2*np.pi+angle
    
    return angle

def get_lane_bins(line):
    angle = get_lane_angle(line)
    angle = np.clip(angle,-1,1) 
    mybin = int(round(angle/0.125))+8
    return mybin,angle


#Ok lets analyze our lanes whether we are correct or now

590 1640 3


In [4]:
count = 0
for sample in lane_dataset:
    image,lines,st_lines,idx = sample
    colors = {0:(255,0,0),1:(0,255,0),2:(0,0,255),3:(255,255,255)}
    
    for j,l in enumerate(lines):
        my_line = l.astype(int)
        bin_no,_ = get_lane_bins(l) #get the bin no
        line_length = len(l)
        for i,p in enumerate(my_line):
            cv2.circle(image, (p, int(y_col[i])), i+1, colors[j], -1)
            if(i==(line_length-10)):
                cv2.putText(image,str(bin_no),org=(p,int(y_col[i])),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=3,color=(255,255,0),thickness=3)



  
    image = cv2.resize(image,(224,224)) #Just checking if the size is explainable. Yes its explainable.
    cv2.imshow("Frame",image)
    k=cv2.waitKey(100)
    if(k==ord('q')):
        break

cv2.destroyAllWindows()

#The thing finally works, I am so happy :)))))))))))))))))
#https://huggingface.co/docs/transformers/model_doc/detr we will try to use this

### Now we will try with image transformation since we are going to fine tune it.

In [1]:
#First we will download a DETR and run it. with our image and check it. Do we need to transform it.
from transformers import DetrConfig, DetrForObjectDetection

/home/akshayd/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/akshayd/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
config = DetrConfig(use_pretrained_backbone=True)
model = DetrForObjectDetection(config) #Run this tommorow and send in our image and then check if it works. If works
#then proceed to start creating a dataloader. Before that also try checking can we convert these images into mp4, so it would be good to load these 
#images as video. (It does not seem possible. Hence the best way is to create a mapper. )

In [10]:
import torch
torch.save(model,"DETR.pt")

In [17]:
model

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [ ]:
batch = np.array([image])
batch = torch.tensor(batch,dtype=torch.float32)
model.forward(batch)

In [22]:
batch.shape

torch.Size([1, 590, 1640, 3])

In [24]:
batch_new = batch.permute(0,3,1,2)

In [25]:
model.forward(batch_new)

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-6.3639e-02,  2.0978e-01, -1.3594e-02],
         [-2.2060e-02,  2.0408e-01, -5.2721e-02],
         [-1.6233e-01,  3.8920e-01,  1.0316e-01],
         [-5.4664e-02,  1.9075e-01,  2.1684e-01],
         [ 7.1715e-02,  1.0766e-01,  1.1517e-01],
         [-8.7906e-02,  3.2916e-01, -7.2125e-02],
         [-7.9909e-02,  1.2397e-01,  9.7978e-02],
         [-7.3927e-02,  2.0328e-01, -1.8262e-02],
         [-1.4205e-02,  2.3487e-01, -1.8988e-01],
         [ 6.9962e-02,  3.0018e-01,  5.2837e-02],
         [ 1.7633e-02,  2.2801e-01,  2.9483e-02],
         [-2.6397e-01,  4.5569e-01,  1.7397e-01],
         [-2.0628e-02,  2.2165e-01, -3.7679e-03],
         [ 2.4040e-02,  3.5502e-01, -1.6690e-02],
         [-2.1083e-01,  6.9795e-02, -1.1733e-03],
         [-1.0057e-01,  2.5838e-01, -1.5519e-01],
         [-1.1348e-01,  2.7037e-01, -4.0372e-02],
         [-1.5310e-02,  1.9243e-01,  8.7596e-02],
         [-1.3983e-01,  3.0542e-01,  7.5

In [26]:
model.forward(batch_new)

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-1.0409e-01,  3.0347e-01,  1.7064e-01],
         [-1.7359e-01,  3.0291e-01, -9.9562e-02],
         [ 1.0967e-01,  8.0928e-02,  4.6577e-02],
         [-2.8326e-02,  5.2057e-01,  1.7896e-02],
         [ 7.7806e-02,  1.0271e-01,  9.0881e-02],
         [-7.5765e-03,  2.6573e-01, -6.8448e-02],
         [ 2.9670e-02,  2.4518e-01,  5.0516e-03],
         [-7.8846e-02,  2.9826e-01,  3.8472e-02],
         [-1.1144e-01,  4.0235e-01,  3.4955e-03],
         [-2.6486e-01,  1.5899e-01, -4.8622e-02],
         [-1.1788e-01,  4.0312e-01, -2.2572e-01],
         [-8.6811e-02,  4.9291e-01,  9.7079e-03],
         [-2.6443e-01,  3.6598e-01,  9.9030e-04],
         [-2.5379e-01,  3.0000e-01,  1.2241e-01],
         [-1.1531e-01,  2.7365e-01,  3.9403e-02],
         [-5.0652e-02,  2.6599e-01, -3.1841e-02],
         [-4.2623e-04,  1.1811e-01, -1.3334e-01],
         [-7.8858e-02,  3.8310e-01, -5.3388e-02],
         [ 8.3395e-02,  2.0351e-01, -4.5

In [27]:
#The model works. We can use it.

In [28]:
new_model = torch.load('DETR.pt')

In [29]:
new_model.forward(batch_new)

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[ 4.0469e-02,  3.4859e-01, -5.3608e-02],
         [-8.4185e-02,  2.2027e-01, -1.0010e-02],
         [-1.0076e-01,  2.3683e-01,  2.1164e-01],
         [ 6.6972e-02,  3.0610e-01,  8.4740e-02],
         [-5.3085e-02,  1.8248e-01,  5.9662e-02],
         [ 3.7948e-03,  2.5617e-01,  1.1193e-01],
         [-2.1734e-01,  3.6573e-01,  1.6362e-01],
         [-7.3840e-02,  2.6624e-01,  1.5213e-01],
         [ 3.3350e-02,  3.4539e-01, -1.1917e-01],
         [-2.0332e-01,  5.7449e-02, -6.3264e-02],
         [-3.7631e-03,  2.5420e-01,  7.4495e-02],
         [ 1.2699e-01,  9.8932e-02,  2.9267e-01],
         [ 2.4591e-02,  3.2339e-01,  2.1284e-02],
         [-9.5998e-02,  1.1063e-01,  8.8845e-02],
         [-7.5928e-02,  1.2851e-01,  7.6373e-02],
         [-8.5582e-02,  1.7313e-01,  2.3267e-02],
         [ 1.6603e-02,  2.6746e-01, -7.4343e-02],
         [ 2.1701e-02,  4.5244e-01, -7.8405e-02],
         [-1.9337e-01,  1.8948e-01, -1.0

In [ ]:
#the above procedures work. But we have a problem of training. It seems that the speed is important for lane detection. So we got to use another model :)

In [4]:
import pickle
import torch
import cv2
from torch.utils.data import Dataset, DataLoader
import numpy as np
#https://pytorch.org/hub/ultralytics_yolov5/

# with open("CUDatamap_line2.pkl","wb") as f:
#     pickle.dump(data_line_map,f) #Store the datamap for further processig
# with open("CUDatamap_coef.pkl","wb") as f:
#     pickle.dump(data_coef_map,f) #Store the datamap for further processig

with open("CUDatamap_line2.pkl","rb") as f:
    data_line_map = pickle.load(f)
with open("CUDatamap_stline2.pkl","rb") as f:
    data_stline_map = pickle.load(f)

with open("CUDatamap_coef.pkl","rb") as f:
    data_coef_map = pickle.load(f)



y_col = np.linspace(400,580,19)

class LaneDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_line_map,data_stline_map,scale=None):
        
        self.new_data_keys = list(data_line_map.keys())
        self.data_line_map = data_line_map
        self.data_stline_map = data_stline_map
        self.scale = scale
        image_loc = self.new_data_keys[0]
        image = cv2.imread(image_loc)
        height,width,channel = image.shape
        self.height = height
        self.width = width
        

  

    def __len__(self):
        return len(self.new_data_keys)
    
    


    def get_lane_angle(self,myline):
        global y_col #its actually row
        row = y_col[-1]
        col = myline[-1]
        
        new_col = col - self.width//2
        new_row = row + self.height

        angle = -np.arctan2(new_col,new_row)
        # if angle<0:
        #     angle = 2*np.pi+angle
        
        return angle

    def get_lane_bins(self,stline):
        angle = get_lane_angle(stline)
        angle = np.clip(angle,-1,1) 
        mybin = int(round(angle/0.125))+8 #17 bins -1 to 1 as 0 to 17 

        return mybin,angle    

        
    def get_regression_confidence_mat(self,stlines,lines):
        regression_mat = np.zeros((19,17),dtype=np.float32) #19 rows and 17 for 17 bins
        confidence_mat = np.zeros((17,),dtype=np.float32)
        for stline,line in zip(stlines,lines):
            mybin,angle = self.get_lane_bins(stline=stline)
            regression_mat[:,mybin] = line
            confidence_mat[mybin] = 1
        return regression_mat.flatten(),confidence_mat

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_loc = self.new_data_keys[idx]
        lines = self.data_line_map[image_loc] #Since
        image = cv2.imread(image_loc)
        stlines = self.data_stline_map[image_loc]

        
        visualize_bundle = (image,lines,stlines,idx) #use only for visualization
        

        image_tensor = torch.tensor(image,dtype=torch.float32)/255 #convert to tensor and normalise
        image_tensor2 = image_tensor.permute(2,0,1) #bring the channel to front

        regression_mat,confidence_mat  = self.get_regression_confidence_mat(stlines,lines)
        regression_mat_tensor = torch.tensor(regression_mat,dtype=torch.float32)
        confidence_mat_tensor = torch.tensor(confidence_mat,dtype=torch.float32)

        #print(image_loc,sum(regression_mat))
        

        return image_tensor2,regression_mat_tensor,confidence_mat_tensor,visualize_bundle

In [5]:
lane_dataset = LaneDataset(data_line_map=data_line_map,data_stline_map=data_stline_map)

In [43]:
count=0
for input,reg_label,confidence_label,_ in lane_dataset:
    for i in range(17):
        print(reg_label.reshape(19,17)[:,i])
    print("Confidence \n ",confidence_label)
    count+=1
    print("-----------------\n-------------------\n")
    if count==5:
        break

driver_161_90frame/06030819_0755.MP4/00000.jpg 72997.12228393555
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 960.6061, 1023.1119, 1085.8741, 1148.8555, 1212.0187, 1275.3263,
        1338.7412, 1402.2261, 1465.7435, 1529.2565, 1592.7273, 1656.1189,
        1719.3939, 1782.5151, 1845.4452, 1908.1469, 1970.5828, 2032.7156,
        2094.5081])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 827.2851,  845.2495,  863.2636,  881.3212,  899.4160,  917.5417,
         935.6921,  953.8610,  972.0419,  990.2288, 1008.4153, 1026.5952,
        1044.7621, 1062.9099, 1081.0323, 1099.1230, 1117.1758, 1135.1843,
        1153.1425])
tensor([0., 0., 0., 0., 0.

In [24]:
input.shape

torch.Size([3, 590, 1640])

In [25]:
for i in range(17):
    print(reg_label.reshape(19,17)[:,i])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 960.6061, 1023.1119, 1085.8741, 1148.8555, 1212.0187, 1275.3263,
        1338.7412, 1402.2261, 1465.7435, 1529.2565, 1592.7273, 1656.1189,
        1719.3939, 1782.5151, 1845.4452, 1908.1469, 1970.5828, 2032.7156,
        2094.5081])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 827.2851,  845.2495,  863.2636,  881.3212,  899.4160,  917.5417,
         935.6921,  953.8610,  972.0419,  990.2288, 1008.4153, 1026.5952,
        1044.7621, 1062.9099, 1081.0323, 1099.1230, 1117.1758, 1135.1843,
        1153.1425])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor

In [6]:
count = 0
for input,reg_label,confidence_label,sample in lane_dataset:
    image,lines,st_lines,idx = sample
    colors = {0:(255,0,0),1:(0,255,0),2:(0,0,255),3:(255,255,255)}
    
    for j,l in enumerate(lines):
        my_line = l.astype(int)
        bin_no,_ = get_lane_bins(l) #get the bin no
        line_length = len(l)
        for i,p in enumerate(my_line):
            cv2.circle(image, (p, int(y_col[i])), i+1, colors[j], -1)
            if(i==(line_length-10)):
                cv2.putText(image,str(bin_no),org=(p,int(y_col[i])),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=3,color=(255,255,0),thickness=3)



  
    
    cv2.imshow("Frame",image)
    k=cv2.waitKey(100)
    if(k==ord('q')):
        break

cv2.destroyAllWindows()

#The thing finally works, I am so happy :)))))))))))))))))
#https://huggingface.co/docs/transformers/model_doc/detr we will try to use this

In [47]:
lines

[array([692.86671224, 653.62724994, 614.3665179 , 575.08815153,
        535.79578626, 496.49305751, 457.18360071, 417.87105129,
        378.55904467, 339.25121628, 299.95120153, 260.66263587,
        221.3891547 , 182.13439347, 142.90198759, 103.69557249,
         64.51878359,  25.37525632, -13.73137389]),
 array([735.74703557, 724.36613272, 712.94722973, 701.49564157,
        690.01668319, 678.51566955, 666.99791562, 655.46873636,
        643.93344673, 632.39736169, 620.8657962 , 609.34406523,
        597.83748373, 586.35136668, 574.89102901, 563.46178571,
        552.06895173, 540.71784204, 529.41377158]),
 array([766.83783173, 776.0340337 , 785.31114942, 794.65880185,
        804.06661399, 813.5242088 , 823.02120927, 832.54723836,
        842.09191906, 851.64487433, 861.19572717, 870.73410054,
        880.24961742, 889.73190078, 899.17057361, 908.55525887,
        917.87557956, 927.12115863, 936.28161907])]

In [48]:
confidence_label

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.])

In [54]:
for i in range(17):
    print(reg_label.reshape(19,17)[:,i])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([766.8378, 776.0341, 785.3112, 794.6588, 804.0666, 813.5242, 823.0212,
        832.5472, 842.0919, 851.6449, 861.1957, 870.7341, 880.2496, 889.7319,
        899.1706, 908.5552, 917.8755, 927.1212, 936.2816])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([735.7470, 72

In [ ]:
#The above values confirm the dataset creation. And its values and confidences. The next step is model design